# Clustering

The main class for clustering in scikit-learn is [sklearn.cluster](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster).

The algorithms discussed here are KMeans, 

The functions and their documentation are given below:

- [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans)
- [MiniBatchKMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html#sklearn.cluster.MiniBatchKMeans)

In [2]:
from sklearn import cluster, metrics, datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import numpy as np

## KMeans

The KMeans algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares. This algorithm requires the number of clusters to be specified. **It scales well to large number of samples** and has been used across a large range of application areas in many different fields.

The K-means algorithm aims to choose centroids that minimise the inertia, or within-cluster sum of squared criterion:

$$\sum_{i=0}^{n} \min_{\mu_j\in C} (\mid\mid x_j-\mu_i\mid\mid^2)$$

In very high-dimensional spaces, Euclidean distances tend to become inflated (this is an instance of the so-called “curse of dimensionality”). **Running a dimensionality reduction algorithm such as PCA prior to k-means clustering can alleviate this problem and speed up the computations.**

K-means is equivalent to the expectation-maximization algorithm with a small, all-equal, diagonal covariance matrix.

#### Parallelization

A parameter can be given to allow K-means to be run in parallel, called `n_jobs`. Giving this parameter a positive value uses that many processors (default: 1). A value of -1 uses all available processors, with -2 using one less, and so on.

### Handwritten digit recognition using KMeans
Preparing the data:

In [ ]:
np.random.seed(2017-12-30)
digits = datasets.load_digits()
data = scale(digits.data)

n_samples, n_features = data.shape
n_digits = len(np.unique(digits.target))
labels = digits.target

sample_size = 300

In [ ]:
km = cluster.KMeans(init='k-means++', n_clusters=n_digits, n_init=10, n_jobs = -1)
km.fit(data)
km.labels_

Using `init='k-means++'` initializes the centroids to be (generally) distant from each other, leading to provably better results than random initialization.

In [7]:
metrics.confusion_matrix(digits.target, km.labels_)

array([[  0, 177,   0,   0,   0,   0,   0,   0,   1,   0],
       [  0,   0,  96,   1,   0,   0,  58,   0,   0,  27],
       [  5,   1,  23,   0,   0,   0,   4,   1,   1, 142],
       [160,   0,   7,   8,   0,   0,   0,   7,   0,   1],
       [  0,   0,   2,   2,   0,   8,   7,   4, 158,   0],
       [ 39,   0,   2, 137,   2,   0,   0,   0,   2,   0],
       [  0,   1,   5,   0, 174,   0,   1,   0,   0,   0],
       [  0,   0,   1,   1,   0,  18,   3, 156,   0,   0],
       [ 46,   1, 105,   6,   2,   0,  11,   2,   0,   1],
       [144,   0,   2,   4,   0,   3,  19,   8,   0,   0]])

Digits 1,3,4,7,8, and 9 have been clustered into separate columns as can be seen from the columns of the confusion matrix. Other points are distributed among clusters.

In [8]:
km = cluster.KMeans(init='random', n_clusters=n_digits, n_init=10, n_jobs = -1)
km.fit(data)
metrics.confusion_matrix(digits.target, km.labels_)

array([[  0,   0,   1,   0, 177,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,  27,  96,  58,   1,   0],
       [  0,   1,   0,   0,   0,  46,  38,   6,  80,   6],
       [  0,   8,   0,   0,   0,   1,   8,   0,   6, 160],
       [  9,   4, 159,   0,   0,   0,   2,   6,   1,   0],
       [  0,   7,   2,   4,   0,   0,  19,   3,  70,  77],
       [  0,   0,   0, 175,   1,   0,   4,   1,   0,   0],
       [ 17, 150,   0,   0,   0,   0,   1,   3,   8,   0],
       [  0,   2,   0,   3,   0,   0,  99,  15,  11,  44],
       [  3,   8,   0,   0,   0,   0,   3,  19,   1, 146]])

In [12]:
pca = PCA(n_components=n_digits).fit(data)
km = cluster.KMeans(init=pca.components_, n_clusters=n_digits, n_init=1, n_jobs = -1)
km.fit(data)
metrics.confusion_matrix(digits.target, km.labels_)

array([[  0,   0,   1, 177,   0,   0,   0,   0,   0,   0],
       [  0,   0,  98,   0,   0,   0,  57,  27,   0,   0],
       [  0,   1,  25,   0,   0,  91,   4,  48,   0,   8],
       [  0,   7,   9,   0,   0,  13,   0,   1,   1, 152],
       [163,   5,   5,   0,   0,   0,   7,   0,   1,   0],
       [  1,   0,   1,   0,   1,  18,   0,   0, 139,  22],
       [  0,   0,   5,   1, 174,   0,   1,   0,   0,   0],
       [  1, 159,  10,   0,   0,   8,   0,   0,   1,   0],
       [  0,   2,  99,   1,   2,   9,   9,   0,   6,  46],
       [  0,  10,   2,   0,   0,   0,  20,   0,   4, 144]])

Here we have provided the centers exactly in the form of an array. Therefore, we do not need to run KMeans multiple times because the result would not change, hence `n_init=1`. 

### Mini Batch KMeans

The `MiniBatchKMeans` is a variant of the `KMeans` algorithm which uses mini-batches to reduce the computation time, while still attempting to optimise the same objective function. Mini-batches are subsets of the input data, randomly sampled in each training iteration. 

`MiniBatchKMeans` converges faster than `KMeans`, but the quality of the results is reduced. In practice this difference in quality can be quite small, as shown in the example and cited reference.

In [13]:
# Generate sample data
np.random.seed(0)
batch_size = 45
centers = [[1, 1], [-1, -1], [1, -1]]
n_clusters = len(centers)
X, labels_true = datasets.make_blobs(n_samples=3000, centers=centers, cluster_std=0.7)

mbk = cluster.MiniBatchKMeans(init='k-means++', n_clusters=3, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose=0)
mbk.fit(X)

MiniBatchKMeans(batch_size=45, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=3,
        n_init=10, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

#### More will be added later